In [33]:
import pandas as pd

In [34]:
%pwd

'/Users/rahulsinghindata/AI-ML/ML_PROJECT_END_TO_END_17_07_2025'

In [35]:
import os
os.chdir("../")

In [36]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/winequality-red.csv'

In [ ]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [ ]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [ ]:
#Entity: Data Validation Configuration
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataValidationConfig:
    """Data Validation Configuration Data Class."""
    root_dir: Path
    source_url: str
    unzip_data_dir: Path
    all_Schema: dict

In [ ]:
#configuration manager

from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH,
        schema_filepath: str = SCHEMA_FILE_PATH):

        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        self.schema = read_yaml(str(schema_filepath))

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.data_validation

        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            unzip_data_dir=config.unzip_data_dir,
            all_Schema=schema
        )
        
        return data_validation_config


In [ ]:
import os
from src.mlproject import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = data.columns.tolist()

            all_schema = self.config.all_Schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}\n")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}\n")
            return validation_status
    
        except Exception as e:
            logger.exception(f"Exception occurred during data validation: {e}")
            raise e
        

In [ ]:
#Pipeline

try:
    config = ConfigurationManager()
    print("Config keys:", config.config.keys())  # Debug: print config keys
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    print(f"Error: {e}")
    raise e

[2025-07-19 01:58:04,651 - INFO - common - Successfully read YAML file at config/config.yaml]
[2025-07-19 01:58:04,653 - INFO - common - Successfully read YAML file at params.yaml]
[2025-07-19 01:58:04,655 - INFO - common - Successfully read YAML file at schema.yaml]
[2025-07-19 01:58:04,655 - INFO - common - Directory created at: artifacts]
Directory created at: artifacts


BoxKeyError: "'ConfigBox' object has no attribute 'data_validation'"